In [1]:
# Tutorial 4: Multi-population recurrent network (with BioNet)

In [2]:
# 一般来说，用户可以为细胞和细胞类型分配任何自定义参数，并将其用作创建连接和运行模拟的属性

In [13]:
import numpy as np
from bmtk.builder.networks import NetworkBuilder
from bmtk.builder.auxi.node_params import positions_columinar, xiter_random

# 构建类似小鼠初级视觉皮层柱的结构，且沿着柱的中心有40个兴奋性Scnnla细胞和10个抑制性PV细胞
net = NetworkBuilder("V1")

# 添加兴奋性Scnnla细胞（N = 40 × 2 = 80）
net.add_nodes(
    N=80, pop_name='Scnn1a',  # 细胞的个数和名称
    positions=positions_columinar(N=80, center=[0, 50.0, 0], max_radius=30.0, height=100.0),  # 中心位置
    rotation_angle_yaxis=xiter_random(N=80, min_x=0.0, max_x=2*np.pi),  # y轴旋转角度
    rotation_angle_zaxis=xiter_random(N=80, min_x=0.0, max_x=2*np.pi),  # z轴旋转角度
    tuning_angle=np.linspace(start=0.0, stop=360.0, num=80, endpoint=False),  # 细胞内部的“调谐角（turning angle）”
                                                                              # 用np.linspace函数生成从0到360等差的80个数（endpoint=False表示不包含360）
    # 宏的细胞类型参数，对全部的80个神经元都有效
    location='VisL4',
    ei='e',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    model_processing='aibs_perisomatic',
    dynamics_params='472363762_fit.json',
    morphology='Scnn1a_473845048_m.swc'
)


# 添加抑制性PV细胞（N = 10 × 2 = 20），参数意义同Scnnla
net.add_nodes(
    N=20, pop_name='PV',
    positions=positions_columinar(N=20, center=[0, 50.0, 0], max_radius=30.0, height=100.0),
    rotation_angle_yaxis=xiter_random(N=20, min_x=0.0, max_x=2*np.pi),
    rotation_angle_zaxis=xiter_random(N=20, min_x=0.0, max_x=2*np.pi),
    location='VisL4',
    ei='i',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    model_processing='aibs_perisomatic',
    dynamics_params='472912177_fit.json',
    morphology='Pvalb_470522102_m.swc'
)

# 打印
print('Success!')

Success!


In [14]:
# 构建点神经元（Point Neuron），使用整合发放模型（Integrate-and-Fire Model）
# 点神经元没有rotation等属性
net.add_nodes(
    N=200, pop_name='LIF_exc',
    positions=positions_columinar(N=200, center=[0, 50.0, 0], min_radius=30.0, max_radius=60.0, height=100.0),
    tuning_angle=np.linspace(start=0.0, stop=360.0, num=200, endpoint=False),
    location='VisL4',
    ei='e',
    model_type='point_process',
    model_template='nrn:IntFire1',
    dynamics_params='IntFire1_exc_1.json'
)

net.add_nodes(
    N=100, pop_name='LIF_inh',
    positions=positions_columinar(N=100, center=[0, 50.0, 0], min_radius=30.0, max_radius=60.0, height=100.0),
    location='VisL4',
    ei='i',
    model_type='point_process',
    model_template='nrn:IntFire1',
    dynamics_params='IntFire1_inh_1.json'
)

# 打印
print('Success!')

Success!


In [15]:
# 建立连接
# 注意，对于不同的模型类型、source神经元的兴奋与否、抑制与否，都要使用不同的突触类型与参数
# 利用距离和调谐角属性确定兴奋-兴奋连接矩阵，为此构造函数dist_tuning_connector

import random
import math

# 对于用户构造的每个自定义函数，都必须有source和target参数
# d是距离，d_weight是距离的权重，nsyn是连接数，t是调谐角的权重
# max和 min表示数值取这两个端点值中间的随机数
def dist_tuning_connector(source, target, d_weight_min, d_weight_max, d_max, t_weight_min, t_weight_max, nsyn_min, nsyn_max):
    
    # 当起始节点与目标节点相同时返回None以防止自身连接
    if source['node_id'] == target['node_id']:
        return None
    
    r = np.linalg.norm(np.array(source['positions']) - np.array(target['positions']))  # 起始节点与目标节点之间的距离
    
    # 计算距离的权重
    if r > d_max:  # 如果节点之间的距离大于我们想要的最大距离，则令这条连接上的权重为0（即舍弃这条连接）
        dw = 0.0  # 赋0和None意义一样，都表示舍弃这条连接，但是None无法在下面的if条件中比大小，因此只能赋值0
    else:
        # 对于不舍弃的连接，计算其权重，公式如下
        t = r / d_max
        dw = d_weight_max * (1.0 - t) + d_weight_min * t
    
    # 若计算出来的权重太低，则丢弃
    if dw <= 0:
        return None

    # 如果细胞中有tuning_angle属性，则计算调谐角的权重，若没有，则令调谐角权重等于距离权重，即令tw = dw
    if 'tuning_angle' in source and 'tuning_angle' in target:

        # 0°-180°和180°-360°没差，因此建模的时候只用0°-180°就行了
        delta_tuning = math.fmod(abs(source['tuning_angle'] - target['tuning_angle']), 180.0)  # 利用math.fmod函数计算节点间角度差绝对值与180°的取余结果

        # 90°-180°需要翻转，然后归一化为0-1
        delta_tuning = delta_tuning if delta_tuning < 90.0 else 180.0 - delta_tuning
        
        # 计算权重
        t = delta_tuning / 90.0
        tw = t_weight_max * (1.0 - t) + t_weight_min * t
    else:
        tw = dw

    # 若计算出来的权重太低，则丢弃
    if tw <= 0:
        return None

    # 通过将权重视为连接概率来过滤掉节点
    # 若生成的0-1间随机数大于调谐角权重，则舍弃该连接
    if random.random() > tw:
        return None

    # 给这条连接添加突触个数
    # 利用random.randint函数生成nsyn_min（我们期望的最小突触个数）和nsyn_max（我们期望的最大突触个数）之间的随机整数（注意是整数）
    return random.randint(nsyn_min, nsyn_max)

# 打印
print('Function defined success!')

Function defined success!


In [16]:
# 添加边
net.add_edges(
    source={'ei': 'e'}, target={'pop_name': 'Scnn1a'},
    connection_rule=dist_tuning_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 0.34, 'd_max': 300.0, 't_weight_min': 0.5,
                       't_weight_max': 1.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=5e-05,
    weight_function='gaussianLL',
    weight_sigma=50.0,
    distance_range=[30.0, 150.0],
    target_sections=['basal', 'apical'],
    delay=2.0,
    dynamics_params='AMPA_ExcToExc.json',
    model_template='exp2syn'
)

net.add_edges(
    source={'ei': 'e'}, target={'pop_name': 'LIF_exc'},
    connection_rule=dist_tuning_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 0.34, 'd_max': 300.0, 't_weight_min': 0.5,
                       't_weight_max': 1.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.0019,
    weight_function='gaussianLL',
    weight_sigma=50.0,
    delay=2.0,
    dynamics_params='instantaneousExc.json',
    model_template='exp2syn'
)

# 打印
print('Success!')

Success!


In [17]:
from bmtk.builder.auxi.edge_connectors import distance_connector

# 同理，构建其他连接
# 考虑到其他细胞没有调谐角参数，因此我们不使用自定义的dist_tuning_connector函数，而是调用封装好的distance_connector函数

# 抑制-抑制连接
net.add_edges(
    source={'ei': 'i'}, target={'ei': 'i', 'model_type': 'biophysical'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 1.0, 'd_max': 160.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.0002,
    weight_function='wmax',
    distance_range=[0.0, 1e+20],
    target_sections=['somatic', 'basal'],
    delay=2.0,
    dynamics_params='GABA_InhToInh.json',
    model_template='exp2syn'
)

net.add_edges(
    source={'ei': 'i'}, target={'ei': 'i', 'model_type': 'point_process'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 1.0, 'd_max': 160.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.001,
    weight_function='wmax',
    delay=2.0,
    dynamics_params='instantaneousInh.json',
    model_template='exp2syn'
)


# 抑制-兴奋连接
net.add_edges(
    source={'ei': 'i'}, target={'ei': 'e', 'model_type': 'biophysical'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 1.0, 'd_max': 160.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.0001,
    weight_function='wmax',
    distance_range=[0.0, 50.0],
    target_sections=['somatic', 'basal', 'apical'],
    delay=2.0,
    dynamics_params='GABA_InhToExc.json',
    model_template='exp2syn'
)

net.add_edges(
    source={'ei': 'i'}, target={'ei': 'e', 'model_type': 'point_process'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 1.0, 'd_max': 160.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.009,
    weight_function='wmax',
    delay=2.0,
    dynamics_params='instantaneousInh.json',
    model_template='exp2syn'
)


# 兴奋-抑制连接
net.add_edges(
    source={'ei': 'e'}, target={'pop_name': 'PV'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 0.26, 'd_max': 300.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.004,
    weight_function='wmax',
    distance_range=[0.0, 1e+20],
    target_sections=['somatic', 'basal'],
    delay=2.0,
    dynamics_params='AMPA_ExcToInh.json',
    model_template='exp2syn'
)

net.add_edges(
    source={'ei': 'e'}, target={'pop_name': 'LIF_inh'},
    connection_rule=distance_connector,
    connection_params={'d_weight_min': 0.0, 'd_weight_max': 0.26, 'd_max': 300.0, 'nsyn_min': 3, 'nsyn_max': 7},
    syn_weight=0.006,
    weight_function='wmax',
    delay=2.0,
    dynamics_params='instantaneousExc.json',
    model_template='exp2syn'
)

# 打印
print('Success!')

Success!


In [18]:
# 构建与保存
net.build()
net.save_nodes(output_dir='sim_ch04/network')
net.save_edges(output_dir='sim_ch04/network')

# 打印
print('Finish!')

Finish!


In [19]:
# 构建外部网络，该网络由虚拟单元组成，形成一个前馈网络到我们的V1，它将在模拟过程中提供输入
# 这个外部网络就是LGN，LGN是V1第四层细胞的主要输入
from bmtk.builder.networks import NetworkBuilder

lgn = NetworkBuilder('LGN')
lgn.add_nodes(
    N=500,
    pop_name='tON',
    potential='exc',
    model_type='virtual'
)

# 打印
print('Success!')

Success!


In [20]:
# 定义函数以确定突触个数
def select_source_cells(sources, target, nsources_min=10, nsources_max=30, nsyns_min=3, nsyns_max=12):
    total_sources = len(sources)
    nsources = np.random.randint(nsources_min, nsources_max)
    selected_sources = np.random.choice(total_sources, nsources, replace=False)
    syns = np.zeros(total_sources)
    syns[selected_sources] = np.random.randint(nsyns_min, nsyns_max, size=nsources)
    return syns

# 打印
print('Function Success!')

Function Success!


In [21]:
# 添加边
lgn.add_edges(
    source=lgn.nodes(), target=net.nodes(pop_name='Scnn1a'),
    iterator='all_to_one',  # 默认为'one_to_one'，可以手动设置成'one_to_all'或者'all_to_one'，to前后分别是source和target节点
    connection_rule=select_source_cells,
    connection_params={'nsources_min': 10, 'nsources_max': 25},
    syn_weight=1e-03,
    weight_function='wmax',
    distance_range=[0.0, 150.0],
    target_sections=['basal', 'apical'],
    delay=2.0,
    dynamics_params='AMPA_ExcToExc.json',
    model_template='exp2syn'
)

lgn.add_edges(
    source=lgn.nodes(), target=net.nodes(pop_name='PV1'),
    connection_rule=select_source_cells,
    connection_params={'nsources_min': 15, 'nsources_max': 30},
    iterator='all_to_one',
    syn_weight=0.015,
    weight_function='wmax',
    distance_range=[0.0, 1.0e+20],
    target_sections=['somatic', 'basal'],
    delay=2.0,
    dynamics_params='AMPA_ExcToInh.json',
    model_template='exp2syn'
)

lgn.add_edges(
    source=lgn.nodes(),  target=net.nodes(pop_name='LIF_exc'),
    connection_rule=select_source_cells,
    connection_params={'nsources_min': 10, 'nsources_max': 25},
    iterator='all_to_one',
    syn_weight= 0.07,
    weight_function='wmax',
    delay=2.0,
    dynamics_params='instantaneousExc.json',
    model_template='exp2syn'
)

lgn.add_edges(
    source=lgn.nodes(),  target=net.nodes(pop_name='LIF_inh'),
    connection_rule=select_source_cells,
    connection_params={'nsources_min': 15, 'nsources_max': 30},
    iterator='all_to_one',
    syn_weight=0.05,
    weight_function='wmax',
    delay=2.0,
    dynamics_params='instantaneousExc.json',
    model_template='exp2syn'
)

# 打印
print('Success!')

Success!


In [22]:
# 构建与保存
lgn.build()
lgn.save_nodes(output_dir='sim_ch04/network')
lgn.save_edges(output_dir='sim_ch04/network')

# 打印
print('Finish!')

Finish!


In [23]:
# 设置环境
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(
    base_dir='sim_ch04',
    config_file='config.json',
    network_dir='sim_ch04/network',
    tstop=3000.0, dt=0.1,
    report_vars=['v'],     # Record membrane potential (default soma)
    include_examples=True,    # Copies components files
    compile_mechanisms=True   # Will try to compile NEURON mechanisms
)

# 打印
print('Env Success!')

/home/tianyan/sim_ch04/components/mechanisms
Mod files: "modfiles/modfiles/CaDynamics.mod" "modfiles/modfiles/Ca_HVA.mod" "modfiles/modfiles/Ca_LVA.mod" "modfiles/modfiles/Ih.mod" "modfiles/modfiles/Im.mod" "modfiles/modfiles/Im_v2.mod" "modfiles/modfiles/Kd.mod" "modfiles/modfiles/K_P.mod" "modfiles/modfiles/K_T.mod" "modfiles/modfiles/Kv2like.mod" "modfiles/modfiles/Kv3_1.mod" "modfiles/modfiles/Nap.mod" "modfiles/modfiles/NaTa.mod" "modfiles/modfiles/NaTs.mod" "modfiles/modfiles/NaV.mod" "modfiles/modfiles/SK.mod" "modfiles/modfiles/vecevent.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../modfiles/CaDynamics.mod
 -> NMODL ../modfiles/Ca_HVA.mod
 -> NMODL ../modfiles/Ca_LVA.mod
 -> NMODL ../modfiles/Ih.mod
 -> NMODL ../modfiles/Im.mod
 -> NMODL ../modfiles/Im_v2.mod
 -> NMODL ../modfiles/Kd.mod
 -> NMODL ../modfiles/K_P.mod
 -> NMODL ../modfiles/K_T.mod
 -> NMODL ../modfiles/Kv2like.mod
 -> NMODL ../modfiles/Kv3_1.mod
 -> NMODL ../modfiles/Nap

Translating CaDynamics.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/CaDynamics.c
Translating Ca_HVA.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Ca_HVA.c
Translating Ca_LVA.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Ca_LVA.c
Thread Safe
Thread Safe
Thread Safe
Translating Ih.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Ih.c
Translating Im_v2.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Im_v2.c
Translating Im.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Im.c
Thread Safe
Thread Safe
Thread Safe
Translating Kd.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Kd.c
Thread Safe
Translating K_P.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/K_P.c
Translating K_T.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/K_T.c
Thread Safe
Thread Safe
Translating Kv2like.mod into /home/tianyan/sim_ch04/components/mechanisms/x86_64/Kv2like.c
Thread Safe
Translating Kv3_1.

 -> Compiling K_P.c
 -> Compiling K_T.c
 -> Compiling Kv2like.c
 -> Compiling Kv3_1.c
 -> Compiling Nap.c
 -> Compiling NaTa.c
 -> Compiling NaTs.c
 -> Compiling NaV.c
 -> Compiling SK.c
 -> Compiling vecevent.c
 => LINKING shared library ./libnrnmech.so
 => LINKING executable ./special LDFLAGS are:    -pthread
Successfully created x86_64/special
Env Success!


/opt/conda/bin/nrnivmodl:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set


In [31]:
# 先前的步骤中，我们在添加权重时使用了函数gaussianLL和wmax
# wmax是一个内置函数，使用参数weight_max的值分配给给定的边类型
# 当创建兴奋-兴奋连接的时候，我们想在需要的时候利用Tuning_angle参数确定两个连接之间的突触强度，因此自定义一个函数gaussianLL
# 下面是构建函数gaussianLL的代码

import math
from bmtk.simulator.bionet.pyfunction_cache import add_weight_function

def gaussianLL(edge_props, source, target):  # 参数edge_props表示连接的属性
    src_tuning = source['tuning_angle']
    tar_tuning = target['tuning_angle']
    w0 = edge_props["syn_weight"]
    sigma = edge_props["weight_sigma"]

    delta_tuning = abs(abs(abs(180.0 - abs(float(tar_tuning) - float(src_tuning)) % 360.0) - 90.0) - 90.0)
    return w0 * math.exp(-(delta_tuning / sigma) ** 2)

add_weight_function(gaussianLL)

# 打印
print('Finish!')

Finish!


In [25]:
'''
    每次模拟前会调整权重，不同的运行之间可以改变功能
    只需使用文本编辑器打开edge_type.csv文件并更改weight_function列
    用户就可以使用现有网络并动态地重新调整权重
'''

'\n    每次模拟前会调整权重，不同的运行之间可以改变功能\n    只需使用文本编辑器打开edge_type.csv文件并更改weight_function列\n    用户就可以使用现有网络并动态地重新调整权重\n'

In [32]:
# 运行
from bmtk.simulator import bionet

conf = bionet.Config.from_json('sim_ch04/config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

# 打印
print('Finish!')

OSError: [Errno 16] Device or resource busy: '.nfs00000000344ea2ae00000e33'

In [28]:
# 分析
from bmtk.analyzer.spike_trains import plot_raster, plot_rates_boxplot

plot_raster(config_file='sim_ch04/config.json', group_by='pop_name')

IndexError: list index out of range